In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "ilchulyukin_laba3") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [2]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
sc

<SparkContext master=yarn appName=ilchulyukin_laba3>

In [4]:
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer,StringIndexer,OneHotEncoder
import pyspark.sql.functions as f_
from pyspark.sql.types import FloatType, StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
import json
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### Чтение источников
- laba03_train.csv - содержатся факты покупки (колонка purchase) пользователями (колонка user_id) телепередач (колонка item_id). 
- laba03_test.csv — тестовый датасет без указанного целевого признака purchase, который вам и предстоит предсказать.
- laba03_items.csv — дополнительные данные по items. В данном файле много лишней или ненужной информации, так что задача её фильтрации и отбора ложится на вас.
- laba03_views_programmes.csv - Дополнительный файл по просмотрам передач 

In [5]:
s = StructType([StructField('user_id',StringType(),True)
               ,StructField('item_id',StringType(),True)
               ,StructField('purchase',IntegerType(),True)]) 

### train

In [6]:
train_csv = spark.read.csv('/labs/slaba03/laba03_train.csv', s, sep = ",", header = True)
print(train_csv.count())
train_csv.show(5)
# Распределение по целевой переменной
train_csv.groupBy("purchase").count().show()

5032624
+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows

+--------+-------+
|purchase|  count|
+--------+-------+
|       1|  10904|
|       0|5021720|
+--------+-------+



### test

In [7]:
test_csv  = spark.read.csv('/labs/slaba03/laba03_test.csv', s, sep = ",", header = True)
print(test_csv.count())
test_csv.show(5)

2156840
+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



### laba03_items.csv — дополнительные данные по items.
Поля в файле, на которых хотелось бы остановиться:
- item_id — primary key. Соответствует item_id в предыдущем файле.
- content_type — тип телепередачи (1 — платная, 0 — бесплатная). Вас интересуют платные передачи.
- title — название передачи, текстовое поле.
- year — год выпуска передачи, число.
- genres — поле с жанрами передачи, разделёнными через запятую.

In [8]:
items_csv = spark.read.csv('/labs/slaba03/laba03_items.csv', sep="\t", header = True)\
                 .select('item_id'
                        ,'content_type'
                        ,f_.regexp_replace('title',r'[^\pL0-9\p{Space}]','' ).alias('title')
                        ,f_.coalesce(f_.col('year'),f_.lit(0)).cast('Integer').alias('year')
                        ,f_.coalesce(f_.regexp_replace(
                                f_.regexp_replace(
                                    f_.regexp_replace(f_.col('genres'), r'[^\pL0-9,\p{Space}]','')
                                        ,' ','_')
                                            ,',',' '),f_.lit('n/a')).alias('genres')


                        )\
                    .select("*"
                    ,f_.when(f_.col('year') <= f_.lit(1964), f_.lit(1964))\
                       .when((f_.col('year') >= f_.lit(1965))&(f_.col('year') <= f_.lit(1975)), f_.lit(1975))\
                       .when((f_.col('year') >= f_.lit(1976))&(f_.col('year') <= f_.lit(1985)), f_.lit(1985))\
                       .when((f_.col('year') >= f_.lit(1986))&(f_.col('year') <= f_.lit(1996)), f_.lit(1996))\
                       .when((f_.col('year') >= f_.lit(1997))&(f_.col('year') <= f_.lit(2004)), f_.lit(2004))\
                       .when((f_.col('year') >= f_.lit(2005))&(f_.col('year') <= f_.lit(2008)), f_.lit(2008))\
                       .when((f_.col('year') >= f_.lit(2006))&(f_.col('year') <= f_.lit(2010)), f_.lit(2010))\
                              .otherwise(f_.round(f_.col('year').cast('integer'),-1)).alias('year_grp')
                           )


print(items_csv.count())
items_csv.show(5)

635568
+-------+------------+--------------------+----+-------+--------+
|item_id|content_type|               title|year| genres|year_grp|
+-------+------------+--------------------+----+-------+--------+
|  65667|           1|на пробах только ...|2013|Эротика|    2010|
|  65669|           1|скуби ду эротичес...|2011|Эротика|    2010|
|  65668|           1|горячие девочки д...|2011|Эротика|    2010|
|  65671|           1|соблазнительницы ...|2011|Эротика|    2010|
|  65670|           1|секретные сексмат...|2010|Эротика|    2010|
+-------+------------+--------------------+----+-------+--------+
only showing top 5 rows



### laba03_views_programmes.csv — Дополнительный файл по просмотрам передач 
 ... с полями::
- ts_start — время начала просмотра.
- ts_end — время окончания просмотра.
- item_type — тип просматриваемого контента:
        live — просмотр "вживую", в момент показа контента в эфире.
        pvr — просмотр в записи, после показа контента в эфире.

In [9]:
views_programmes_csv = spark.read.csv('/labs/slaba03/laba03_views_programmes.csv', sep=",", header = True)
print(views_programmes_csv.count())
views_programmes_csv.show(5)

20845607
+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
+-------+-------+----------+----------+---------+
only showing top 5 rows



### item_id из items_csv нужны только те, которые есть в tran или test

In [10]:
items_cache = train_csv.select('item_id').union(test_csv.select('item_id')).distinct()\
                       .join(items_csv,['item_id'],'left').cache()
print(items_cache.count())
items_cache.show(5)
items_cache.groupBy("title").count().show(5)
items_cache.groupBy("genres").count().show(5)
items_cache.groupBy('year_grp').count().orderBy('year_grp').show(50)

3704
+-------+------------+-------------------+----+--------------------+--------+
|item_id|content_type|              title|year|              genres|year_grp|
+-------+------------+-------------------+----+--------------------+--------+
| 100140|           1|            поездка|2014|             Комедии|    2010|
| 100263|           1|          фортитьюд|2015|Ужасы Детективы Т...|    2020|
| 100735|           1|сенлоран стиль этоя|2014|Артхаус Драмы Мел...|    2010|
|   1159|           1|              залив|2012|Ужасы Триллеры Фа...|    2010|
|   2136|           1|          проклятая|2009|Ужасы Триллеры За...|    2010|
+-------+------------+-------------------+----+--------------------+--------+
only showing top 5 rows

+--------------------+-----+
|               title|count|
+--------------------+-----+
|гидра затерянный ...|    1|
|              риф 3d|    3|
|   храбрый портняжка|    1|
|            не горюй|    1|
|      волшебная сила|    1|
+--------------------+-----+
only sh

### Фичи User_id

In [11]:
user_ = train_csv.join(items_cache.select("item_id", "title", "year", "genres", "year_grp"),["item_id"], "left")

In [12]:
user_.select("user_id","item_id").distinct().count()

5032624

####  Программы
- количество предложенных программ пользователю
- количество купленных пользователем программ
- доля купленных программ

In [13]:
user_progs = user_.groupBy("user_id").agg(
                                          f_.count("item_id").alias("user_items_cnt")
                                         ,f_.sum("purchase").alias("purchase_user_items_cnt")
                                         ,(f_.sum("purchase")/f_.count("item_id")).alias("user_rate")
                                         )

####  Жанры
- genres_buy_cnt - количество купленных пользователем жанров
- top_3_genres_buy_txt -  top 3 купленных жанров
- genre - все жанры

In [14]:
genre_token = Tokenizer(inputCol = "genres", outputCol = "genre").transform(user_)

user_genre_ = genre_token\
   .select("user_id", f_.explode("genre").alias("genres"), "genre", "purchase")\
   .groupBy("user_id", "genres").agg(f_.count("*").alias("genre_count")
                                    ,f_.sum("purchase").alias("genre_buy_count")
                                    ,f_.max("genre").alias("genre")
                                     )\
   .select("user_id", "genres", "genre"
          ,f_.sum(f_.col("genre_buy_count")).over(Window.partitionBy("user_id")).alias("genres_buy_cnt")
           
          ,f_.row_number().over(Window.partitionBy("user_id")\
                                .orderBy(f_.col("genre_count").cast("Integer").desc())).alias("genres_rn")
          ,f_.row_number().over(Window.partitionBy("user_id")\
                                .orderBy(f_.col("genre_buy_count").cast("Integer").desc())).alias("genres_buy_rn")
                 )

user_genre_buy = user_genre_.where("genres_buy_rn < 4")\
                        .groupBy("user_id").agg(f_.max("genre").alias("genre")
                                               ,f_.max("genres_buy_cnt").alias("genres_buy_cnt")
                                               ,f_.collect_set("genres").alias("top_3_genres_buy_txt")
                            )

In [15]:
user_ftr = user_progs.join(user_genre_buy, ["user_id"], "left").cache()

In [16]:
user_ftr.count()

1941

In [17]:
user_ftr.show(5)

+-------+--------------+-----------------------+--------------------+--------------------+--------------+--------------------+
|user_id|user_items_cnt|purchase_user_items_cnt|           user_rate|               genre|genres_buy_cnt|top_3_genres_buy_txt|
+-------+--------------+-----------------------+--------------------+--------------------+--------------+--------------------+
| 867363|          2569|                      1|3.892565200467107...|[фильмы, фантасти...|             4|[мистические, фан...|
| 882935|          2585|                      2|7.736943907156673E-4|[эротика, триллер...|             5|[зарубежные, дете...|
| 889974|          2548|                      0|                 0.0|[русские_мультфил...|             0|[полнометражные, ...|
| 891250|          2624|                      0|                 0.0|[фильмы, фантасти...|             0|[фантастика, дете...|
| 902451|          2600|                      1|3.846153846153846E-4|[эротика, комедии...|             2|[драмы

In [18]:
genre_buy_conv = CountVectorizer(inputCol = "top_3_genres_buy_txt", outputCol = "top_3_genres_buy_v", binary = True)
genre_conv = CountVectorizer(inputCol = "genre", outputCol = "genres_v", binary = True)
user_features = Pipeline(
                    stages=[genre_buy_conv, genre_conv] 
                        )

user_features_final =user_features.fit(user_ftr).transform(user_ftr) 

In [19]:
user_features_final.columns

['user_id',
 'user_items_cnt',
 'purchase_user_items_cnt',
 'user_rate',
 'genre',
 'genres_buy_cnt',
 'top_3_genres_buy_txt',
 'top_3_genres_buy_v',
 'genres_v']

In [20]:
user_features_final.show(5)

+-------+--------------+-----------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|user_id|user_items_cnt|purchase_user_items_cnt|           user_rate|               genre|genres_buy_cnt|top_3_genres_buy_txt|  top_3_genres_buy_v|            genres_v|
+-------+--------------+-----------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
| 867363|          2569|                      1|3.892565200467107...|[фильмы, фантасти...|             4|[мистические, фан...|(83,[14,15,32],[1...|(74,[0,10,18],[1....|
| 882935|          2585|                      2|7.736943907156673E-4|[эротика, триллер...|             5|[зарубежные, дете...|(83,[0,4,18],[1.0...|(74,[0,1,2,3],[1....|
| 889974|          2548|                      0|                 0.0|[русские_мультфил...|             0|[полнометражные, ...|(83,[11,13,72],[1...|(74,[8,9

###   Фичи  Items
- группы годов выпуска программ
    - StringIndexer - индексация строк по количеству вхождений
    - OneHotEncoder - векторизация категорий (индексов)

В PySpark есть специальный список стоп-слов, доступный через loadDefaultStopWords(language)

In [21]:
items_cache.where("genres is null").count()

0

In [22]:
items_cache.columns

['item_id', 'content_type', 'title', 'year', 'genres', 'year_grp']

In [23]:
# Год выпуска
year_group_index  = StringIndexer(inputCol = "year_grp", outputCol = "yearIndex")
year_group_vector = OneHotEncoder(inputCol = "yearIndex", outputCol = "year_group_v")

transform_features_pipeline = Pipeline(
                    stages=[
                            year_group_index,
                            year_group_vector
                           ] 
                                )
transform_features_pipeline_models = transform_features_pipeline.fit(items_cache)                                               
item_data = transform_features_pipeline_models.transform(items_cache)

In [24]:
train = train_csv.join(item_data,['item_id'],'left')

In [25]:
train.select("user_id").count()

5032624

In [26]:
# Покупательный рейтинг фильма
item_rate = train.groupBy('item_id')\
                 .agg(f_.sum(f_.col('purchase')).alias('item_buy')
                     ,(f_.sum(f_.col('purchase').cast('integer'))/f_.count('*')).alias('item_rate')
                            ) 

In [27]:
# Рейтинг просмотров зрителя
user_views= views_programmes_csv.join(items_cache.select('item_id','content_type'),['item_id'],'left')\
        .select('*', (f_.col('ts_end')-f_.col('ts_start')).alias('time'))\
        .groupBy('user_id')\
        .agg( f_.count('*').alias('views_cnt')
             ,f_.avg(f_.col('time')).alias('views_avg_time')
             ,f_.sum(f_.when(f_.col('item_type')==f_.lit('live'),f_.lit(1)).otherwise(f_.lit(0))
                    ).alias('views_live_cnt')
            )

In [28]:
views_programmes_csv.join(items_cache.select('item_id'),['item_id'],'left').show(15)

+-------+-------+----------+----------+---------+
|item_id|user_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|7101053|      0|1491409931|1491411600|     live|
|7101054|      0|1491412481|1491451571|     live|
|7101054|      0|1491411640|1491412481|     live|
|6184414|      0|1486191290|1486191640|     live|
|4436877|    257|1490628499|1490630256|     live|
|7489015|   1654|1493434801|1493435401|     live|
|7489023|   1654|1493444101|1493445601|     live|
|6617053|   1654|1489186156|1489200834|     live|
|6438693|   1654|1487840070|1487840433|     live|
|6526859|   1654|1488705452|1488706154|     live|
|6526754|   1654|1488532396|1488532895|      pvr|
|6239098|   1654|1486732011|1486732410|     live|
|6438763|   1654|1488305761|1488307286|      pvr|
|7489013|   1654|1493433301|1493434201|     live|
|6317094|   1654|1486829784|1486830389|     live|
+-------+-------+----------+----------+---------+
only showing top 15 rows



In [29]:
views_programmes_csv.columns

['user_id', 'item_id', 'ts_start', 'ts_end', 'item_type']

### Фичи  short list
 - purchase_user_items_cnt - количество купленных программ пользователем
 - item_buy - количество пользователей, купивших программу
 - user_rate - рейтинг покупок пользователя
 - item_rate - рейтинг покупаемости прграммы 
 - genres_buy_cnt - количество жанров, купленных пользователем
 - top_3_genres_buy_v - TOP 3 жанра, купленных пользователем
 - genres_v - все жанры, купленные пользователем
 - year_group_v - группа года выпуска программы 
 - views_avg_time - среднее время просмотра программ пользователем
 - views_live_cnt - количество просмотров пользователем программ

In [30]:
ftr_short_list=[
 "purchase_user_items_cnt"
,"item_buy"
,'user_rate'
,'item_rate'  
,'genres_buy_cnt'
,'top_3_genres_buy_v'
,'genres_v'
,'year_group_v'
    
,'views_avg_time'
,'views_live_cnt'             ]
assembler = VectorAssembler(inputCols = ftr_short_list, outputCol = "features")

In [31]:
%%time
train_data_ = train.alias('t')\
    .join(user_features_final.alias("f"),['user_id'],'left')\
    .join(item_rate.alias('r'),['item_id'],'left')\
    .join(user_views.alias('v'),['user_id'],'left')\
    .fillna( { 
                'views_avg_time':0,
                'views_live_cnt':0
                    } )
train_data=assembler.transform(train_data_)\
    .select('user_id','item_id',f_.col('purchase').cast('integer').alias('purchase'),'features')\
        .repartition(50)

CPU times: user 7.95 ms, sys: 0 ns, total: 7.95 ms
Wall time: 93.8 ms


In [32]:
!hdfs dfs -rm -r -skipTrash tmp_data_lab03

Deleted tmp_data_lab03


In [33]:
%%time
train_data.write.parquet('tmp_data_lab03',mode = 'overwrite')
train_data = spark.read.parquet('tmp_data_lab03')

CPU times: user 9.57 ms, sys: 777 µs, total: 10.3 ms
Wall time: 56.9 s


In [34]:
train_sample = train_data.sampleBy("purchase", fractions = {0: 0.8, 1: 0.8}, seed = 1868)
valid_sample = train_data.join(train_sample, on=["item_id",'user_id'], how = "leftanti")

### LogReg

In [35]:
%%time
evaluator = BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="purchase"
                                         ,metricName = 'areaUnderROC')
logreg = LogisticRegression(featuresCol = 'features', labelCol = "purchase", maxIter = 130, regParam = 0.05)
logreg_model = logreg.fit(train_sample)
predictions_v = logreg_model.transform(valid_sample)
predictions_t = logreg_model.transform(train_sample)
roc_v = evaluator.evaluate(predictions_v)
roc_t = evaluator.evaluate(predictions_t)
print(roc_v, roc_t)

0.9117878371015103 0.9190757313103809
CPU times: user 33.6 ms, sys: 4.26 ms, total: 37.9 ms
Wall time: 1min 18s


In [36]:
print(ftr_short_list, roc_t, '(130 / 0.05)', ' ')

['purchase_user_items_cnt', 'item_buy', 'user_rate', 'item_rate', 'genres_buy_cnt', 'top_3_genres_buy_v', 'genres_v', 'year_group_v', 'views_avg_time', 'views_live_cnt'] 0.9190757313103809 (130 / 0.05)  


### G-Boost

In [37]:
%%time
evaluator = BinaryClassificationEvaluator(rawPredictionCol = "probability", labelCol = "purchase"
                                         ,metricName = 'areaUnderROC')
gbt = GBTClassifier(featuresCol = 'features'
                   ,labelCol = "purchase", maxDepth = 4, maxBins = 50, minInstancesPerNode = 3)
gbt_model = gbt.fit(train_sample)
predictions_ = gbt_model.transform(valid_sample)
roc_= evaluator.evaluate(predictions_)
print(roc_)

0.9219773846198998
CPU times: user 36.6 ms, sys: 27.7 ms, total: 64.3 ms
Wall time: 5min 30s


# Test

In [38]:
test_data_ = test_csv.join(item_data,['item_id'],'left').alias('t')\
    .join(user_features_final.alias("f"),['user_id'],'left')\
    .join(item_rate.alias('i'),['item_id'],'left')\
    .join(user_views.alias('v'),['user_id'],'left')\
    .fillna( { 
                'views_avg_time':0,
                'views_live_cnt':0                    } )
test_data = assembler.transform(test_data_)\
                     .select('user_id','item_id',f_.col('purchase').cast('integer').alias('purchase'),'features')

In [39]:
test_data.show(5)

+-------+-------+--------+--------------------+
|user_id|item_id|purchase|            features|
+-------+-------+--------+--------------------+
| 867363| 100263|    null|(171,[0,1,2,3,4,1...|
| 867363| 100735|    null|(171,[0,1,2,3,4,1...|
| 867363|   1159|    null|(171,[0,1,2,3,4,1...|
| 867363|  74605|    null|(171,[0,2,4,19,20...|
| 867363|  81824|    null|(171,[0,1,2,3,4,1...|
+-------+-------+--------+--------------------+
only showing top 5 rows



In [40]:
!hdfs dfs -rm -r -skipTrash test_data_lab03

Deleted test_data_lab03


In [41]:
%%time
test_data.write.parquet('test_data_lab03',mode='overwrite')
test_data = spark.read.parquet('test_data_lab03')
print(test_csv.count(),test_data.count())

2156840 2156840
CPU times: user 3.56 ms, sys: 7.42 ms, total: 11 ms
Wall time: 55.2 s


In [42]:
@f_.udf(ArrayType(FloatType()))
def vector_to_list(v):
    return v.toArray().tolist()

### LogReg

In [43]:
predictions_logreg = logreg_model.transform(test_data)

In [44]:
output_ = predictions_logreg\
    .select('user_id','item_id', vector_to_list('probability').getItem(1).alias('purchase'))\
    .orderBy(f_.col('user_id').asc(),f_.col('item_id').cast("Integer").asc()).coalesce(1)

### G-Boost

In [45]:
predictions_gbt = gbt_model.transform(test_data)

In [46]:
output_ = predictions_gbt\
    .select('user_id','item_id', vector_to_list('probability').getItem(1).alias('purchase'))\
    .orderBy(f_.col('user_id').asc(),f_.col('item_id').cast("Integer").asc()).coalesce(1)

### Output file

In [47]:
output_.where("purchase = 'NaN'").show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
+-------+-------+--------+



In [48]:
%%time
output_.orderBy(f_.col('user_id').asc(),f_.col('item_id').cast("Integer").asc())\
.write.csv('lab03_.csv',mode = 'overwrite',sep = ',',header = True)

CPU times: user 8.34 ms, sys: 787 µs, total: 9.12 ms
Wall time: 29.6 s


In [49]:
! hdfs dfs -ls /user/ilya.chulyukin/lab03_.csv

Found 2 items
-rw-r--r--   3 ilya.chulyukin ilya.chulyukin          0 2022-11-06 01:03 /user/ilya.chulyukin/lab03_.csv/_SUCCESS
-rw-r--r--   3 ilya.chulyukin ilya.chulyukin   52015790 2022-11-06 01:03 /user/ilya.chulyukin/lab03_.csv/part-00000-07e30d77-9835-4a60-bac2-c071d44d0ebc-c000.csv


In [50]:
! hadoop fs -mv lab03_.csv/part-00000-*.csv lab03_.csv/lab03.csv

In [51]:
! rm -r lab03.csv

In [52]:
! hdfs dfs -get -f hdfs://spark-master-1.newprolab.com:8020/user/ilya.chulyukin/lab03_.csv/lab03.csv